# PASSO 00 IMPORTS

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_absolute_error
import boto3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


import requests
from datetime import datetime
import boto3

import mlflow

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("mlflow bitcoin")


<Experiment: artifact_location='mlflow-artifacts:/783824972131025485', creation_time=1737229390822, experiment_id='783824972131025485', last_update_time=1737229390822, lifecycle_stage='active', name='mlflow bitcoin', tags={}>

# PASSO 01 DATA DESCRIPTION

In [2]:
def get_historical_data(coin, days):
    url = f'https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency=usd&days={days}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        prices = data['prices']
        df = pd.DataFrame(prices, columns=["Timestamp", "Price (USD)"])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
        return df
    except Exception as e:
        st.error(f"Erro ao buscar dados históricos: {e}")
        return pd.DataFrame()

In [3]:
historical_data = get_historical_data('bitcoin',days=120)
historical_data.to_parquet('../data/raw/historical_data.parquet')

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar os dados (supondo que já tenha sido coletado)
df_principal = pd.read_parquet('../data/raw/historical_data.parquet')

# FEATURE ENGINEERING

In [5]:
df = df_principal.copy()
df.head()

,Timestamp,Price (USD)
0,2024-10-15,66049.994927
1,2024-10-16,66962.219945
2,2024-10-17,67647.544148
3,2024-10-18,67328.105270
4,2024-10-19,68465.619207


In [6]:
df = df.rename(columns={"Timestamp":"timestamp","Price (USD)":"price (usd)"})
df

,timestamp,price (usd)
0,2024-10-15 00:00:00,66049.994927
1,2024-10-16 00:00:00,66962.219945
2,2024-10-17 00:00:00,67647.544148
3,2024-10-18 00:00:00,67328.105270
4,2024-10-19 00:00:00,68465.619207
...,...,...
116,2025-02-08 00:00:00,96558.456310
117,2025-02-09 00:00:00,96558.237625
118,2025-02-10 00:00:00,96548.578654
119,2025-02-11 00:00:00,97399.981614


In [7]:
df['day_of_week'] = df['timestamp'].dt.dayofweek


# Extrair o número do dia no mês
df['day_of_month'] = df['timestamp'].dt.day

In [8]:
# Média móvel de 7 dias (curto prazo)
df['moving_avg_7'] = df['price (usd)'].rolling(window=7).mean()

# Média móvel de 30 dias (médio prazo)
df['moving_avg_30'] = df['price (usd)'].rolling(window=30).mean()

# Média móvel de 90 dias (longo prazo)
df['moving_avg_90'] = df['price (usd)'].rolling(window=90).mean()

In [9]:
# Calcular a variação percentual diária (retorno)
df['daily_return'] = df['price (usd)'].pct_change()

# Calcular a variação absoluta diária
df['daily_return_abs'] = df['price (usd)'].diff()

In [10]:
df['volatility'] = df['daily_return'].rolling(window=30).std()

In [11]:
# Função para calcular o RSI
def calculate_rsi(data, window):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Calcular o RSI de 14 dias
df['RSI_14'] = calculate_rsi(df['price (usd)'], window=14)

In [12]:
# Média exponencial de 7 dias
df['ema_7'] = df['price (usd)'].ewm(span=7, adjust=False).mean()

# Média exponencial de 30 dias
df['ema_30'] = df['price (usd)'].ewm(span=30, adjust=False).mean()

# Variable Filtering

In [13]:
df = df.dropna()

In [14]:
# Lista das variáveis derivadas
derived_columns = [
    'day_of_week',  'day_of_month', 
    'moving_avg_7', 'moving_avg_30', 'moving_avg_90'
]

# Filtrando apenas as variáveis derivadas
df_derived = df[derived_columns]

# Exibindo as primeiras linhas das variáveis derivadas
print(df_derived.head())

    day_of_week  day_of_month  moving_avg_7  moving_avg_30  moving_avg_90
89            6            12  96319.321639   97574.564274   88174.555487
90            0            13  95760.776129   97344.648981   88490.164109
91            1            14  94650.341107   97114.293605   88795.654436
92            2            15  94596.338081   96842.712811   89117.060364
93            3            16  95352.972079   96650.680983   89483.560888


# Machine Learning Modelling

In [15]:
# Separar variáveis independentes (apenas as derivadas)
X_derived = df_derived

# Variável dependente (preço)
y = df['price (usd)']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_derived, y, test_size=0.3, shuffle=False)


In [ ]:
import numpy as np
def treinar_modelo(X_train, y_train):
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    return modelo

def avaliar_modelo(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, rmse, mae, r2

mlflow.autolog()
with mlflow.start_run(log_system_metrics=True):
     # Idealmente, chame isso antes do start_run
    modelo = treinar_modelo(X_train, y_train)
    mse, rmse, mae, r2 = avaliar_modelo(modelo, X_test, y_test)
    mlflow.log_metric('mse',mse)
    mlflow.log_metric('rmse',rmse)  
    mlflow.log_metric('mae',mae)
    mlflow.log_metric('r2',r2)

    print(f'Mean Squared Error (Linear Regression): {mse}')
    print(f'Root Mean Squared Error (RMSE) (Linear Regression): {rmse}')
    print(f'Mean Absolute Error (MAE) (Linear Regression): {mae}')
    print(f'R² (Linear Regression): {r2}')


2025/02/07 15:50:05 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.2, but the installed version is 1.6.0. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/02/07 15:50:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/02/07 15:50:06 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2025/02/07 15:50:06 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2025/02/07 15:50:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/erickfog/ada_tech/Cursos/Produtizacao_de_Modelos_e_MLOPs/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:435: UserWar

Mean Squared Error (Linear Regression): 4449446363.757838
Root Mean Squared Error (RMSE) (Linear Regression): 66704.17051247874
Mean Absolute Error (MAE) (Linear Regression): 60359.569522483725
R² (Linear Regression): -547.0596436795366
🏃 View run clumsy-whale-100 at: http://127.0.0.1:5000/#/experiments/783824972131025485/runs/e4f749b80dfe42ae89c57b1b549ee2d6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/783824972131025485


In [17]:
X_train.columns

Index(['day_of_week', 'day_of_month', 'moving_avg_7', 'moving_avg_30',
       'moving_avg_90'],
      dtype='object')

In [58]:
#mlflow models serve   -m mlflow-artifacts:/783824972131025485/cab2b7bd29ec49bca8a3861d65456dce/artifacts/model   --no-conda   --port 5001

In [21]:
import requests

# Defina os dados aninhados na chave "inputs"
payload = {
    "dataframe_split": {
        "columns": ['day_of_week', 'day_of_month', 'moving_avg_7', 'moving_avg_30', 'moving_avg_90'],
        "data": [[5, 8, 94472.382736, 94472.382736, 94378.144222]]
    }
}

# Envie a requisição POST para o endpoint do MLflow
response = requests.post("http://localhost:5001/invocations", json=payload)

# Imprima a resposta (as predições retornadas pelo modelo)
print(response.json())


ConnectionError: HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7629b8c8c310>: Failed to establish a new connection: [Errno 111] Connection refused'))